In [1]:
include("env.jl")

 Activating environment at `~/Git/advent/2022/CentJulEnv/Project.toml`


   Updating 

registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...


No Changes to `~/Git/advent/2022/CentJulEnv/Project.toml`
No Changes to `~/Git/advent/2022/CentJulEnv/Manifest.toml`


  Resolving package versions...


No Changes to `~/Git/advent/2022/CentJulEnv/Project.toml`
No Changes to `~/Git/advent/2022/CentJulEnv/Manifest.toml`
  Resolving package versions...


No Changes to `~/Git/advent/2022/CentJulEnv/Project.toml`
No Changes to `~/Git/advent/2022/CentJulEnv/Manifest.toml`


In [27]:
# read the assignment list
function get_input(input, k=false)
    # 

    f = open(input)
    l = readlines(f,keep=k)
    close(f)

    line = findall(l.=="")[1]      # finall() returns a 1x1 array, we need the value of this array

    # drawing is the top lines until just before the empty line
    drawing = l[1:line-1] 

    # instructions are after empty line to the end
    instructions = l[line+1:end]

    return drawing, instructions
end



get_input (generic function with 2 methods)

In [28]:
drawing, instructions = get_input("d5_test.txt")

(["    [D]    ", "[N] [C]    ", "[Z] [M] [P]", " 1   2   3 "], ["move 1 from 2 to 1", "move 3 from 1 to 3", "move 2 from 2 to 1", "move 1 from 1 to 2"])

In [29]:
instructions

4-element Array{String,1}:
 "move 1 from 2 to 1"
 "move 3 from 1 to 3"
 "move 2 from 2 to 1"
 "move 1 from 1 to 2"

In [214]:
# map the drawing to a dictionary that we can manipulate
# assign each stack the correct crates
# bottom crate should be first thing in the list

function draw_dicts(drawing)
    # drawing has the pattern [A] [B] [C]
    
    # each item in the stack takes three chars or spaces, with one space inbetween stacks
    # pattern [A] [B]

    # get the number of stacks and stack names
    stacks = split(drawing[end,])

    # initialize the dictionary
    stack_dict = Dict()
    initial_height = length(drawing)-1
    initial_position = 2

    for s in stacks
    
        # initialize each stack dictionary
        stack_dict[s] = Char[]
        position = initial_position + 4*(parse(Int,s) - 1)

        for row = 1:initial_height
            
            if !isspace(drawing[row][position]) #throw out the blank spaces
                push!(stack_dict[s],drawing[row][position])
            end
        end

        # reverse the order after reading the drawing
        stack_dict[s] = reverse(stack_dict[s])
    end
    return stack_dict
end 


draw_dicts (generic function with 1 method)

In [192]:

function rearrange(dict_drawing, instructions)
# move items around according to instructions
# instructions are the form "move X from Y to Z"
# X = number of times to repeat the operation (Int)
# Y = starting stack key 
# Z = ending stack key

    for line in instructions

        split_line = split(line)
        repeats = parse(Int,split_line[2])
        from_stack = split_line[4]
        to_stack = split_line[6]

        for r in 1:repeats
            temp = pop!(dict_drawing[from_stack])
            push!(dict_drawing[to_stack], temp)
        end
        
    end
    return dict_drawing
end

rearrange (generic function with 1 method)

In [246]:
function print_final_msg(final_dict)
    # print the last "top" value from each stack
    # notet that looping through the dict with a (key,value) in dict won't always give you the right order, instead force the order with "iterate"
    
    msg = String[]
    
    for iterate in 1:length(final_dict)
        value = final_dict[string(iterate)]    
        print(value[end])
        #msg *= value[end]
    
    end
    # print(msg)
    # return msg 
end

print_final_msg (generic function with 1 method)

In [236]:
final_dict[string(1)]

1-element Array{Char,1}:
 'C': ASCII/Unicode U+0043 (category Lu: Letter, uppercase)

In [249]:
# run a test
# answer should be CMZ

drawing, instructions = get_input("d5_test.txt");
test_dict =  draw_dicts(drawing);
final_dict = rearrange(test_dict, instructions);
print_final_msg(final_dict);

CMZ

In [250]:
# run another test
# answer should be CMZXY

drawing, instructions = get_input("d5_test2.txt");
test_dict =  draw_dicts(drawing);
final_dict = rearrange(test_dict, instructions);
print_final_msg(final_dict);

CMZXY

In [252]:
# solve day 5 problem 1

drawing, instructions = get_input("day-5-input.txt")
final_dict = rearrange(draw_dicts(drawing), instructions)
print_final_msg(final_dict)

FCVRLMVQP

String[]

## Part 2 - move multiple crates at once 

Don't rearrange the order when moving, i.e. get more clever with push/pop, or use some cut/append strategy



In [325]:
# write a function that will split the stacks and reorder them if it's a model 9000
# the new rearrange function will work for either model of crate mover

function read_instructions(line)
    # instructions are the form "move n from from_stack to to_stack"
       # return instructions
       # n = number of crates to move all at once, INT
       # from_stack = starting stack key, string 
       # to_stack = ending stack key, string
       
       split_line = split(line)
       n = parse(Int, split_line[2])    
       from_stack = split_line[4]
       to_stack = split_line[6]
   
       return n, from_stack, to_stack
   end
   
   function split_stack(from_stack_crates, n, model=9001)
       # input the from_stack and number of crates to move
       # output new from_stack and the crates to move 
       # reverse the order if the model is 9000
       
       crates = from_stack_crates[end-n+1:1:end]
       # crates = from_stack_crates[end:-1:end-n+1] # this is reverse order from expected
   
       if model == 9000
           crates = crates[end:-1:1] #reverse the order
       end
   
       from_stack_crates = from_stack_crates[1:end-n]
       
       return from_stack_crates, crates
   end


function rearrange2(dict_drawing, instructions, model=9001)
    # move items around according to instructions
    # follow pattern from Part2: don't push/pop, split and append (so the order remains the same instead of reversing)
    # if model == 9000 (old model), reverse the crate order to match part1
    
    # instructions are the form "move X from Y to Z"
    # X = number of crates to move all at once
    # Y = starting stack key 
    # Z = ending stack key
    
        for line in instructions
            
            # read instructions for each line
            n, from_stack, to_stack = read_instructions(line)
    
            # split the from stack and append to the to_stack 
            dict_drawing[from_stack], crates = split_stack(dict_drawing[from_stack], n, model)
            dict_drawing[to_stack] = [dict_drawing[to_stack]; crates]                           # double check if this is working...
            
        end
        return dict_drawing
    end

rearrange2 (generic function with 2 methods)

In [321]:
# try some things


drawing, instructions = get_input("d5_test.txt");
test_dict =  draw_dicts(drawing);
test_in = instructions[3]; print(test_in,"\n")
n, f, t = read_instructions(test_in)
print(n,f,t,"\n")
test_stack_crates = test_dict[f]; print(test_stack_crates, "\n")
test_stack_crates, crates = split_stack(test_stack_crates, n); print(test_stack_crates, "\n")

#final_dict = rearrange2(test_dict, instructions);
#print_final_msg(final_dict);

move 2 from 2 to 1
221
['M', 'C', 'D']


['M']


In [326]:
# try some more things
# Answer should be MCD

drawing, instructions = get_input("d5_test.txt");
test_dict =  draw_dicts(drawing);
final_dict = rearrange2(test_dict, instructions);
print_final_msg(final_dict);

MCD

In [332]:
# try some more things
# MCDXY

drawing, instructions = get_input("d5_test2.txt");
test_dict =  draw_dicts(drawing);
final_dict = rearrange2(test_dict, instructions);
print_final_msg(final_dict);

MCDXY

In [331]:
# try some more things
# SMJPBZPVF

drawing, instructions = get_input("d5_test3.txt");
test_dict =  draw_dicts(drawing);
final_dict = rearrange2(test_dict, instructions);
print_final_msg(final_dict);

SMJPBZPVF

In [333]:
# solve day 5 problem 1

drawing, instructions = get_input("day-5-input.txt")
final_dict = rearrange2(draw_dicts(drawing), instructions)
print_final_msg(final_dict)

RWLWGJGFD

String[]